# Socket

Per poter simulare la connessione tra più client a un server, abbiamo bisogno di definire tre file.

1.   Uno per la struttura dei client (con main);
2.   Uno per la gestione delle richieste del client da parte del server; (con run)
3.   Uno per la struttura del server (con main).




---





**1.   CLIENT FILE**





Per gestire la trasmissione delle informazioni andiamo a definere un oggetto della classe Socket che ci permettirà di gestire la comunicazione lato client. Per farlo però dobbiamo importare la librera che ci mette a disposizione la classe:

In [ ]:
import java.net.Socket;
  try (Socket socket = new Socket("localhost", 56789);

Questa è l'istanza che ci permette di creare un'oggetto della classe Socket. Come parametri dovremmo specificare l'indirizzo ip, in questo caso metteremo localhost utilizzando il nostro computer e successivamente la porta in cui avverrà la connessione.

**Come scegliere una porta?**

Puoi usare porte nel range 49152–65535, noto come porte effimere o dinamiche, che sono generalmente non assegnate a servizi noti.

Esempi: 49160, 50000, 60000.

Evita porte già usate da servizi comuni (ad esempio, 3306 per MySQL, 5432 per PostgreSQL, ecc.).



**Gestione della trasmissione**

Per gestire l'input dell'utente che richiede il servizio tramite il client e l'input che invece viene dal server, utilizzaremo la classe BufferedReader e InputStreamReader. Anche queste hanno bisogno per essere utilizzate, di essere importate:

In [ ]:
import java.io.BufferedReader;
import java.io.InputStreamReader;
BufferedReader inFromKeyboard = new BufferedReader(new InputStreamReader(System.in));//lettura dell'input da tastiera
BufferedReader inFromServer = new BufferedReader(new InputStreamReader(socket.getInputStream()));//lettura dell'input restituito dal server

Per gestire l'output dall'utente al server, utilizzereno la classe PrintWriter. Anche qui per essere utilizzato deve essere importato. Per utilizzarlo dobbiamo racchiuderlo in un try e catch come quando leggiamo da un file per interrompere il programma e farci restituire un messaggio di errore se necessario:

In [ ]:
import java.io.PrintWriter;
import java.io.IOException;
//il primo parametro specifica serve a creare un messagio in uscita da uno già esistente, mentre il secondo specifica se lo svuotamento del buffer tramite la flush viene fatto in automatico o no
PrintWriter outToServer = new PrintWriter(socket.getOutputStream(), true)){
            String msg, response;//variabili che conterranno il messaggio da mandare e il messaggio ricevuto dal server
            System.out.println("connection stabilita ("+socket.getInetAddress()+", "+socket.getPort()+")");//stampa del messaggio che ci conferma la stabilità della connessione e le informazioni
            System.out.println("socket creata porta: "+socket.getLocalPort());//specifichiamo la porta dove avviene la connessione
            do {
                msg = inFromKeyboard.readLine();//leggiamo l'input dall'utente
                outToServer.println(msg);//inviamo il messaggio al server
                response = inFromServer.readLine();//riceviamo la risposta dal server
                System.out.println("Server sent: " + response);//stampiamo la risposta a video
            }while(!msg.equalsIgnoreCase("fine"));//finchè l'utente non digita "fine" il client continuerà a rimanere in attesa dell'input dall'utente
            System.out.println("Connection interrupted!");//stampa della fine della connessione
        } catch (IOException e) {
            throw new RuntimeException(e);
        }

**2. GESTIONE CLIENT FILE**



Per gestire le richieste dei client da parte del server, abbiamo bisogno di definire un "modello" di gestione delle richieste da assegnare ad ogni thread, per far si che il server risponda ai vari client in maniera simultanea. Iniziamo nell'importare tutte le librerie comuni con il file client.

In [ ]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.Socket;

Successivamente creiamo la classe ClientHandler, avente gli attributi utili alla gestione.

In [ ]:
public class ClientHandler implements Runnable{
    private Socket socket;
    private BufferedReader inFromClient;
    private PrintWriter outToClient;
    private int contatore;

Il prossimo passo è creare il metodo contruttore che inizializzi gli attributi della classe, acquisendo le informazioni utili dalla lettura del socket assegnato dal server.

In [ ]:
  public ClientHandler(Socket socket) throws IOException {//passaggio del socket da gestire
        this.socket = socket;//asseganzione all'oggetto socket in locale
        inFromClient = new BufferedReader(new InputStreamReader(socket.getInputStream()));//lettura della richiesta del client
        outToClient = new PrintWriter(socket.getOutputStream(), true);//creazione dell'oggetto per l'output verso il client, passando come parametro il canale di trasmissione che è lo stesso del socket e il secondo che rappresenta "auto pulizia" del canale di trasmissione
        contatore = 0;//inizializzazione del contatore che sarà utile a soddisfare la richiesta dei client
    }

In fine bisogna definire l'override del metodo run, che in base agli input dell'utente generà e invierà ad esso una risposta. Utilizziamo try e catch per comunicare un'eventuale errore durante l'operazione da parte del thread gestore.

In [ ]:
@Override
    public void run() {
        String request, msg;//variabili che conterranno il messaggio da mandare e il messaggio ricevuto dal server
        try {
            do{
                request = inFromClient.readLine();//lettura della richiesta del client
                switch(request.toLowerCase()) {//analisi della richiesta e in base a essa generazione della risposta
                    case "conta" -> {//se la richiesta richiede di contare
                        contatore++;//il thread gestore della richiesta al server incrementa il contatore
                        msg = "OK";//genera un messaggio da inviare al clinet che gli dica che l'operazione è andata a buon fine
                    }
                    case "fine" ->//se la richiesta è di chiudere la trasmissione
                        msg = contatore + "";//il thread gestore della richiesta al server prepara la risposta comunicando il contatore incrementato
                    default ->//messaggio di default di richiesta non supportata
                        msg = "BAD REQUEST";
                }
                outToClient.println(msg);//invio al client del messaggio elaborato
            }while(!request.equalsIgnoreCase("fine"));//il thread gestore della richiesta al server continua a rimanere in ascolto finchè il client non richiede la chiusura della trasmissione

            socket.close();//chiude il canale di trasmissione
            inFromClient.close();//chiuda la lettura da client
            outToClient.close();//chiude la scrittura da server
            System.out.println("Connection interrupted!");//comunica la chiusura della connessione
        } catch (IOException e) {
            throw new RuntimeException(e);
        }

**3. SERVER FILE**



Per il file del server andiamo a dichiarare l'oggetto Socket utilizzato dal client per la comunicazione al server, quindi importiamo nuovamente la librearia sui Socket.

In [ ]:
import java.net.Socket;
Socket socket;

Per gestire la trasmissione di informazioni da parte del server dichiaramo la classe ServerSocket. Per dichiararlo ovvimente importiamo anche qui la libreria.

In [ ]:
 try (ServerSocket serverSocket = new ServerSocket(56789);//Come per il client definiamo l'oggetto serverSocket, specificando la porta dove il server si metterà in ascolto

Per gestire più richieste del servizio da parte dei client al server, dichiaramo la classe ExecutorService per far gestire le richiesta da più tread secondo il modello del file precedetente creato. Anche qui per l'esecuzione del file utilizzeremo un try e catch per la stampa di un errore se si presenta un'errore durante l'esecuzione del codice:

In [ ]:
import java.io.IOException;
import java.util.concurrent.ExecutorService;
import java.util.concurrent.Executors;
ExecutorService executorService = Executors.newCachedThreadPool()){//dichiarazione della classe Threadpool già vista

            System.out.println("Server ready on port 56789...");//stampa iniziale del server che comunica la messa in attesa nella porta concordata

            while(true) {//creiamo un ciclo infinito che resta in comunicazione finchè il client non interrompe la connessione
                socket = serverSocket.accept();//accetta la connessione con il client
                executorService.execute(new ClientHandler(socket));//manda in esecuzione un thread che attraverso il modello di gestione, gestiste le richieste del client passando il socket del client richiedente
                System.out.println("New connection estabilished!");//comunica a video che la nuova connessione è stata stabilita
            }

        } catch (IOException e) {
            throw new RuntimeException(e);
        }